# funcX Tutorial

funcX is a Function-as-a-Service (FaaS) platform that enables fire-and-forget execution of Python functions on one or more remote funcX endpoints. 

This tutorial is configured to use a tutorial endpoint hosted by the funcX team. You can setup and use your own endpoint by following the [funcX documentation](https://funcx.readthedocs.io/en/latest/endpoints.html).

## funcX Python SDK

The funcX Python SDK provides programming abstractions for interacting with the funcX service. Before running this tutorial you should first install the funcX SDK as follows:

    $ pip install funcx

The funcX SDK exposes a `FuncXClient` and `FuncXExecutor` for all interactions with the funcX service. In order to use funcX, you must first authenticate using one of hundreds of supported identity provides (e.g., your institution, ORCID, or Google).  As part of the authentication process you must grant permission for funcX to access your identity information (to retrieve your email address), Globus Groups management access (to share functions and endpoints), and Globus Search (to register and discover functions and endpoints).

In [ ]:
from funcx import FuncXClient, FuncXExecutor

fx = FuncXExecutor(FuncXClient())
print("FuncXExecutor : ", fx)

# funcX 101

The following example demonstrates how you can execute a function with the `FuncXExecutor` interface.


### Submitting a function

To invoke a function, you must provide: 

 * the function
 * the `endpoint_id` of the endpoint on which you wish to execute that function
 
Optionally, you may also specify any input arguments to the function. 
    
> **Note**: Here we use the public funcX tutorial endpoint. You may change the `endpoint_id` to the UUID of any endpoint for which you have permission to execute functions.

In [ ]:
# Define the function for remote execution
def hello_world():
    return "Hello World!"

tutorial_endpoint = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint
future = fx.submit(hello_world, endpoint_id=tutorial_endpoint)

print("Submit returned: ", future)

### Getting results

When you `submit` a function for execution (called a `task`), the executor will return an instance of `FuncXFuture` in lieu of the result from the function.  Futures are a common way to reference asynchronous tasks, enabling you to interrogate the future to find the status, results, exceptions, etc without blocking to wait for results.

`FuncXFuture`s returned from the `FuncXExecutor` can be used in the following ways:
* `future.done()` is non-blocking and returns a boolean that indicates whether the task is finished.
* `future.result()` is a blocking call, that returns the result from the remote execution or raises an exception if a task execution failed. 

In [ ]:
# Returns a boolean that indicates task completion
future.done()

In [ ]:
# Waits for the function to complete and returns the task result or exception on failure
future.result()

### Catching exceptions

When a task fails and you try to get its result, the `future` will raise an exception. In the following example, the `ZeroDivisionError` exception is raised when `future.result()` is called.

In [ ]:
def division_by_zero():
    return 42 / 0 # This will raise a ZeroDivisionError

future = fx.submit(division_by_zero, endpoint_id=tutorial_endpoint)

try:
    future.result()
except Exception as exc:
    print("funcX returned an exception: ", exc)

## Functions with arguments

funcX supports registration and invocation of functions with arbitrary arguments and returned parameters. funcX will serialize any `*args` and `**kwargs` when invoking a function and it will serialize any return parameters or exceptions. 

Note: funcX uses standard Python serilaization libraries (e.g., Pickle, Dill). It also limits the size of input arguments and returned parameters to 10 MB. 

The following example shows a function that computes the sum of a list of input arguments.

In [ ]:
def funcx_sum(a, b):
    return a + b

future = fx.submit(funcx_sum, 40, 2, endpoint_id=tutorial_endpoint)
print(f"40 + 2 = {future.result()}")


## Functions with dependencies

funcX requires that functions explictly state all dependencies within the function body. It also assumes that any dependencies (e.g., libraries, modules) are available on the endpoint on which the function will execute.  For example, in the following function, we explictly import the datetime module. 

In [ ]:
def funcx_date():
    from datetime import date
    return date.today()

future = fx.submit(funcx_date, endpoint_id=tutorial_endpoint)

print("Date fetched from endpoint: ", future.result())

## Calling external applications

While funcX is designed to execute Python functions, you can easily invoke external applications that are accessible on the remote enpoint. For example, the following function calls the Linux `echo` command. 

In [ ]:
def funcx_echo(name):
    import os
    return os.popen("echo Hello {} from $HOSTNAME".format(name)).read()

future = fx.submit(funcx_echo, "World", endpoint_id=tutorial_endpoint)

print("Echo output: ", future.result())

## Running functions many times

One of the strengths of funcX is the ease by which you can run functions many times, perhaps with different input arguments.  The following example shows how you can use the Monte Carlo method to estimate pi. 

Specifically, if a circle with radius $r$ is inscribed inside a square with side length $2r$, the area of the circle is $\pi r^2$ and the area of the square is $(2r)^2$. Thus, if $N$ uniformly-distributed points are dropped at random locations within the square, approximately $N\pi/4$ will be inside the circle.


In [ ]:
import time

# function that estimates pi by placing points in a box
def pi(num_points):
    from random import random
    inside = 0   
    
    return 3 # BADD
    for i in range(num_points):
        x, y = random(), random()  # Drop a point randomly within the box.
        if x**2 + y**2 < 1:        # Count points within the circle.
            inside += 1  
    return (inside*4 / num_points)


# execute the function 3 times 
estimates = []
for i in range(3):
    estimates.append(fx.submit(pi, 
                               10**5, 
                               endpoint_id=tutorial_endpoint))

# get the results and calculate the total
total = [future.result() for future in estimates]

# print the results
print("Estimates: {}".format(total))
print("Average: {:.5f}".format(sum(total)/len(estimates)))

# Endpoint operations

You can retrieve information about endpoints including status and information about how the endpoint is configured. 

In [ ]:
from funcx import FuncXClient
fxc = FuncXClient()

fxc.get_endpoint_status(tutorial_endpoint)